In [30]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy import text


In [29]:
sql_con = sa.engine.URL.create(
    drivername = 'mysql+pymysql',
    username = "root",
    password = "mysql4505",
    host = "127.0.0.1",
    database ="globaltech"
)

sql_engine = create_engine(sql_con)

In [10]:
query = ''' select * from merged_data'''

df = pd.read_sql_query(query,sql_engine)
df.head()

,OrderNumber,LineItem,OrderDate,DeliveryDate,CustomerKey,StoreKey,ProductKey,Quantity,CurrencyCode,Gender,...,OpenDate,ProductName,Brand,Color,UnitCostUSD,UnitPriceUSD,SubcategoryKey,Subcategory,CategoryKey,Category
0,366000,1,2016-01-01,2016-01-08,265598,10,1304,1,CAD,Male,...,2015-04-04,Contoso Lens Adapter M450 White,Contoso,White,31.27,68.00,406,Cameras & Camcorders Accessories,4,Cameras and camcorders
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,Male,...,2010-01-01,A. Datum SLR Camera X136 Silver,A. Datum,Silver,141.47,427.00,402,Digital SLR Cameras,4,Cameras and camcorders
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,Male,...,2010-01-01,Fabrikam Microwave 1.5CuFt X1100 Black,Fabrikam,Black,220.64,665.94,803,Microwaves,8,Home Appliances
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,Female,...,2010-01-01,Contoso SLR Camera M146 Orange,Contoso,Orange,148.08,322.00,402,Digital SLR Cameras,4,Cameras and camcorders
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,Female,...,2010-01-01,Adventure Works Laptop8.9 E0890 White,Adventure Works,White,166.20,326.00,301,Laptops,3,Computers


In [43]:
query = text('''DESCRIBE merged_data;''')

df= pd.read_sql_query(query,sql_engine)

df.head()

,Field,Type,Null,Key,Default,Extra
0,OrderNumber,bigint,YES,,None,
1,LineItem,bigint,YES,,None,
2,OrderDate,datetime,YES,,None,
3,DeliveryDate,text,YES,,None,
4,CustomerKey,bigint,YES,,None,


In [54]:
#1. Query to total sales revenue order by year

query = text('''select year(`OrderDate`) as Year, 
                        SUM(UnitPriceUSD)-SUM(UnitCostUSD) as TotalRevenue 
             from merged_data
             group by year(`OrderDate`) 
             ''')
df = pd.read_sql_query(query,sql_engine)
df.head()

,Year,TotalRevenue
0,2016,1.271395e+06
1,2017,1.397547e+06
2,2018,2.369440e+06
3,2019,3.377214e+06
4,2020,1.726735e+06


In [67]:
# 2. query to total sales by years

query = text(''' select year(`OrderDate`) as Year, 
             SUM(Quantity) as Total_sales
             from merged_data
             group by year 
             ''')

df= pd.read_sql_query(query,sql_engine)
df

,Year,Total_sales
0,2016,21761.0
1,2017,24798.0
2,2018,44498.0
3,2019,68440.0
4,2020,34463.0
5,2021,3797.0


In [70]:
#3. Calculate the total number of sales in each store.

query =text(''' Select DISTINCT StoreKey as StoreCode,ProductKey as procductCode,
            Count("OrderNumber") as Number_Sales
            from merged_data
            group by Storekey,ProductKey Order by Number_sales ''')

df = pd.read_sql_query(query,sql_engine)
df

,StoreCode,procductCode,Number_Sales
0,45,1933,1
1,43,341,1
2,49,538,1
3,49,308,1
4,65,1285,1
...,...,...,...
35747,0,453,38
35748,0,454,39
35749,0,419,42
35750,0,444,45


In [94]:
# 4.Find the top 3 products by total sales.

query = text(''' Select Distinct ProductKey as ProductCode,
             Sum(Quantity) as TotalSales,
             ProductName
             from merged_data 
             group by productCode, ProductName
             Order by TotalSales Desc
            limit 3
             ''')

df = pd.read_sql_query(query,sql_engine)
df


,ProductCode,TotalSales,ProductName
0,444,550.0,WWI Desktop PC2.33 X2330 Black
1,458,538.0,WWI Desktop PC1.80 E1800 White
2,434,521.0,Adventure Works Desktop PC2.30 MD230 White


In [100]:
# 5. Find the total sales for each month.

query = text (''' select MONTH (`OrderDate`) as Months,
              SUM(Quantity) as Total_sales
              from merged_data
              group by Month(`Orderdate`)''')

df = pd.read_sql_query(query,sql_engine)
df


,Months,Total_sales
0,1,24018.0
1,2,26911.0
2,3,8876.0
3,4,2001.0
4,5,16525.0
5,6,15538.0
6,7,13925.0
7,8,14751.0
8,9,15688.0
9,10,16258.0


In [118]:
# 6.Find customers who made purchases above the average sales amount.

query = text('''select  Name
             from merged_data 
             Where CustomerKey IN (
                 select CustomerKey 
                 from merged_data 
                 where UnitPriceUSD > (Select AVG(UnitPriceUSD) as AVG from merged_data))
              ''')


df= pd.read_sql_query(query,sql_engine)
df

,Name
0,Tyler Vaught
1,Frank Upchurch
2,Frank Upchurch
3,Joan Roche
4,Joan Roche
...,...
54517,Sofie Molenschot
54518,Donald Woodie
54519,William Rochelle
54520,William Rochelle


In [122]:
# 7. List customers whose total purchase amount exceeds $10,000.

query = text(''' select Distinct CustomerKey,
             SUM(UnitPriceUSD) as Total_Purchases
             from merged_data
             Group by CustomerKey
             Having Total_Purchases > 2500''')

df = pd.read_sql_query(query,sql_engine)
df

,CustomerKey,Total_Purchases
0,980164,4489.62
1,758280,2886.82
2,763246,3199.99
3,209990,3854.71
4,1839627,2638.98
...,...,...
2204,1724519,2554.99
2205,2002239,2600.90
2206,697365,2774.93
2207,76891,2791.10


In [124]:
# 8. Rank each store by total sales amount.

query = text(''' Select StoreKey,
             SUM(UnitPriceUSD) as Total_sales,
             Rank() Over(Order by SUM(UnitPriceUSD) DESC) as Store_Rank 
             from merged_data
             group by StoreKey
             ''')

df = pd.read_sql_query(query,sql_engine)
df



,StoreKey,Total_sales,Store_Rank
0,0,3626167.86,1
1,54,443845.92,2
2,55,438108.87,3
3,9,435729.01,4
4,45,433047.22,5


In [ ]:
# 9. Calculate the total sales amount for each store, but only for the stores that had sales in more than 1000 transactions.

query = text(''' Select StoreKey,
             SUM(UnitPriceUSD) as Total_sales_amount
             from merged_data
             group by StoreKey
             having Count(*) > 1000''')

df = pd.read_sql_query(query,sql_engine)
df

In [128]:
#10.Classify each sale as 'High', 'Medium', or 'Low' based on the sales amount:

# 'High' for sales over $2,500
# 'Medium' for sales between $500 and $2,500
# 'Low' for sales under $500.

query=text(''' select StoreKey, UnitPriceUSD,
           CASE
           WHEN UnitPriceUSD >  2500 THEN "High"
           WHEN UnitPriceUSD BETWEEN 500 AND 2500 THEN "Medium"
           Else "Low"
        
           END as Sales_Level
           from merged_data''')

df = pd.read_sql_query(query,sql_engine)
df

,StoreKey,UnitPriceUSD,Sales_Level
0,10,68.00,Low
1,0,427.00,Low
2,0,665.94,Medium
3,0,322.00,Low
4,0,326.00,Low
...,...,...,...
62879,43,139.00,Low
62880,0,120.00,Low
62881,0,109.99,Low
62882,0,70.13,Low
